In [6]:
import numpy as np;
import matplotlib.pyplot as plt;
import matplotlib as mpl;
import math;
mpl.rcParams['figure.dpi'] = 600;

In [7]:
def getSolution(list):
    c = 0;
    length = len(list[0]);
    answersArray = np.zeros([length-1], dtype=np.float32);
    while(c < length-1):
        column = list[:,c];
        element = 0;
        index = -1;
        count = 0;
        r = 0;
        while(r < len(column)):
            if(column[r] != 0):
                count += 1;
                index = r;
                element = column[r];        
            r += 1;
        if(count == 0):
            print("Empty column! " + str(c)); #Should never happen!
        elif(count == 1):
            answersArray[c] = element*list[index][length-1];        
        c += 1;
    print(answersArray);
    return answersArray;

def isFeasible(list):
    for el in list:
        if(el < 0):
            return False;
    return True;
        
def performMixed(list):
    
    #prepare list
    
    while(not isFeasible(getSolution(list))):
        print("Do stuff");
        #Do stuff
        #1. go through S-values & find where row with a single -1 is present in a column
        #2. go through objective function coefficient and select first column, 
        #   which would result in selection of the row from 1
        #2.b. if such column doesn't exist, go to the next row in 1
        #3. perform elimination
    
    list = maximize(list);

In [8]:
def performMaximization(list):
    list = fillSlack(list);
    list[ len(list)-1  ] *= -1;
    list = maximize(list);
    return list;
    #TODO: analyze results
    
def performMinimization(list):
    list = np.transpose(list);
    list = fillSlack(list);
    list[ len(list)-1  ] *= -1;
    list = maximize(list);
    return list;
    #TODO: analyze results

In [9]:
def printArray(list):
    for sub in list:
        line = "[";
        for el in sub:
            line = line + "%7.1f " % (el);
        line = line + "]";
        print(line);
    print();

In [10]:
def getMinIndex(array):
    index = 0;
    i = 0;
    element = math.inf;
    for el in array:
        if(el < element and el > 0):
            index = i;
            element = el;
        i += 1;
    return index;

In [11]:
def divideArrays(array1, array2): #divides array1 by array2
    temp = np.zeros(len(array1));
    i = 0;
    for x in array2:
        if( x <= 0 ):
            temp[i] = math.inf;
        else:
            temp[i] = array1[i]/array2[i];
        i += 1;
    return temp;

In [12]:
def fillSlack(list, ineq = None):
    nIneq = len(list)-1;
    nVar = len(list[0])-1;
    i = 0;
    while(i < nIneq):
        i += 1; 
        list = np.insert(list, len(list[0])-1, 0, axis= 1 );
    
    i = 0;
    j = nVar;
    while (i < nIneq):
        list[i][(j+i)] = 1;
        if(ineq != None):
            list[i][(j+i)] = ineq[i];
        i += 1;
    return list;

In [21]:
def maximize(list): 
    
    while(min(list[-1]) < 0):
        entryColumn = np.argmin(list[-1]);
        temp = divideArrays(list[0:-1, -1] , list[0:-1, entryColumn]);
        entryRow = np.argmin(temp);#getMinIndex(temp);
        print("Entry point is at: [" + str(entryRow) + ";" + str(entryColumn) + "]");
        list = performElimiation(list, entryRow, entryColumn);
        print("Intermediate result: ")
        printArray(list)
        print();
    return list;

In [14]:
def performElimiation(list, entryRow, entryColumn):
    list[entryRow] = list[entryRow]/list[entryRow][entryColumn];
    i = 0;
    while i < len(list):
        if(i != entryRow):
            list[i] = list[i] - list[i][entryColumn]*list[entryRow]
        i += 1;
    return list;

In [15]:
def fillInMatrix():
    rows = int(input("Number of inequalities: ")) + 1; #
    nVar = int(input("Number of variables: "));
    columns = nVar + 1; # Plus result column
    print("Rows: " + str(rows) + " Columns: " + str(columns));
    inputArray = np.zeros([rows, columns], dtype=np.float32);
    i = 0;
    while(i < rows):
        j = 0;
        if(i == (rows-1)):
            print("Enter values for Objective function: ");
            objFun = i;
        else:
            print("Enter values for Constraints");
        while(j < (nVar+1)):
            if(j != nVar):
                inputArray[i][j] = float(input("Input coefficient of var №" + str(j+1) + ": "));
            elif (i != (rows-1)):
                inputArray[i][columns-1] = float(input("Input resulting variable (b): "));
            elif (i == (rows-1)):
                inputArray[i][columns-1] = 0;
            j += 1;
        print();
        i += 1;
    return inputArray;

In [22]:
print("Fill in initial matrix: ");
inputArray = fillInMatrix();
printArray(inputArray);
#maximize(np.copy(inputArray));
list = performMaximization(np.copy(inputArray));
print("Final matrix: ");
printArray(list);
getSolution(list);

Fill in initial matrix: 
Number of inequalities: 3
Number of variables: 2
Rows: 4 Columns: 3
Enter values for Constraints
Input coefficient of var №1: -1
Input coefficient of var №2: 1
Input resulting variable (b): 11

Enter values for Constraints
Input coefficient of var №1: 1
Input coefficient of var №2: 1
Input resulting variable (b): 27

Enter values for Constraints
Input coefficient of var №1: 2
Input coefficient of var №2: 5
Input resulting variable (b): 90

Enter values for Objective function: 
Input coefficient of var №1: 4
Input coefficient of var №2: 6

[   -1.0     1.0    11.0 ]
[    1.0     1.0    27.0 ]
[    2.0     5.0    90.0 ]
[    4.0     6.0     0.0 ]

Entry point is at: [0;1]
Intermediate result: 
[   -1.0     1.0     1.0     0.0     0.0    11.0 ]
[    2.0     0.0    -1.0     1.0     0.0    16.0 ]
[    7.0     0.0    -5.0     0.0     1.0    35.0 ]
[  -10.0     0.0     6.0     0.0     0.0    66.0 ]


Entry point is at: [2;0]
Intermediate result: 
[    0.0     1.0     